In [1]:
import pandas as pd

In [2]:
p = "mean of diseased pixels in red plane, mean of diseased pixels in green plane, mean of diseased pixels in blue plane, mean of background pixels in red plane, mean of background pixels in green plane, mean of background pixels in blue plane, mean of boundary region pixels of disease in red plane, mean of boundary region pixels of disease in green plane, mean of boundary region pixels of disease in blue plane, mean of core region pixels of disease in red plane, mean of core region pixels of disease in green plane, mean of core region pixels of disease in blue plane, standard deviation of diseased pixels in red plane, standard deviation of diseased pixels in green plane, standard deviation of diseased pixels in blue plane, standard deviation of background pixels in red plane, standard deviation of background pixels in green plane, standard deviation of background pixels in blue plane, standard deviation of boundary region pixels of disease in red plane, standard deviation of boundary region pixels of disease in green plane, standard deviation of boundary region pixels of disease in blue plane, standard deviation of core region pixels of disease in red plane, standard deviation of core region pixels of disease in green plane, standard deviation of core region pixels of disease in blue plane."

In [3]:
q = p.lower().split(",")

In [4]:
d = {'feature_of_data':q}
df = pd.DataFrame(d)

In [5]:
df

,feature_of_data
0,mean of diseased pixels in red plane
1,mean of diseased pixels in green plane
2,mean of diseased pixels in blue plane
3,mean of background pixels in red plane
4,mean of background pixels in green plane
5,mean of background pixels in blue plane
6,mean of boundary region pixels of disease in ...
7,mean of boundary region pixels of disease in ...
8,mean of boundary region pixels of disease in ...
9,mean of core region pixels of disease in red ...


In [6]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
PATH = os.getcwd()
# Define data path
data_path = PATH + '/Datasets'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

['Bacterial leaf blight', 'Brown spot', 'Leaf smut']


In [7]:
import cv2
import numpy as np

def image_masking(img):
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,0.0) # In BGR format
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    contour_info = []
    contours, __ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        contour_info.append((c, cv2.isContourConvex(c), cv2.contourArea(c),))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    
    max_contour = contour_info[0]
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    
    mask_stack = np.dstack([mask]*3)
    mask_stack  = mask_stack.astype('float32') / 255.0
    img = img.astype('float32') / 255.0
    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR)
    masked = (masked * 255).astype('uint8')
    resized_mask= cv2.resize(masked,(256,256),interpolation = cv2.INTER_AREA)
    cv2.imshow("reshaped_img",resized_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#if __name__ == '__main__':
#    filepath = "DSC_0373.JPG"
#    image_masking(filepath)

In [8]:
num_classes = 3

labels_name={'Bacterial leaf blight':0,'Brown spot':1,'Leaf smut':2}

img_data_list=[]
labels_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loading the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        img_data_list.append(input_img)
        labels_list.append(label)

img_data = np.array(img_data_list)
print (img_data.shape)

Loading the images of dataset-Bacterial leaf blight

Loading the images of dataset-Brown spot

Loading the images of dataset-Leaf smut

(120,)


In [9]:
reshaped_img = []
for i in range(120):
    reshaped_img.append(cv2.resize(img_data[i],(256,256)))

In [10]:
for i in range(120):
    print(i)
    image_masking(reshaped_img[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [11]:
rgb = []
for i in range(120):
    rgb.append(cv2.cvtColor(reshaped_img[i],cv2.COLOR_BGR2RGB))
hsv = []
canny = []
for i in range(120):
    hsv.append(cv2.cvtColor(rgb[i],cv2.COLOR_RGB2HSV))
    canny.append(cv2.Canny(rgb[i],100,200))
for i in range(120):
    cv2.imshow("rgb_of_image",rgb[i])
    cv2.imshow("hsv_of_image",hsv[i])
    cv2.imshow("canny",canny[i])
    cv2.waitKey(0)
    

cv2.destroyAllWindows()

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255,   0],
       ...,
       [255, 255,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0]], dtype=uint8)